Proceso de ETL del DataSet steam_games

In [31]:
#Importo las librerias que voy a usar
import pandas as pd
import json
import gzip
import ast

In [32]:
# Tengo que cargar el dataset que voy a utilizar
games = pd.read_json("../Dataset/steam_games.json.gz", compression="gzip", lines=True)
games.head(5)

,publisher,genres,app_name,title,url,release_date,tags,reviews_url,specs,price,early_access,id,developer
0,None,None,None,None,None,None,None,None,None,None,NaN,NaN,None
1,None,None,None,None,None,None,None,None,None,None,NaN,NaN,None
2,None,None,None,None,None,None,None,None,None,None,NaN,NaN,None
3,None,None,None,None,None,None,None,None,None,None,NaN,NaN,None
4,None,None,None,None,None,None,None,None,None,None,NaN,NaN,None


Se debera detectar la cantidad de valores nulos y se debera proceder a eliminarlos

In [33]:
# Con la siguiente funcion elimino las filas que solo tienen valores nulos
games = games.dropna(how='all')
# Y comprobamos si funciona
games.head(3)

,publisher,genres,app_name,title,url,release_date,tags,reviews_url,specs,price,early_access,id,developer
88310,Kotoshiro,"[Action, Casual, Indie, Simulation, Strategy]",Lost Summoner Kitty,Lost Summoner Kitty,http://store.steampowered.com/app/761140/Lost_...,2018-01-04,"[Strategy, Action, Indie, Casual, Simulation]",http://steamcommunity.com/app/761140/reviews/?...,[Single-player],4.99,0.0,761140.0,Kotoshiro
88311,"Making Fun, Inc.","[Free to Play, Indie, RPG, Strategy]",Ironbound,Ironbound,http://store.steampowered.com/app/643980/Ironb...,2018-01-04,"[Free to Play, Strategy, Indie, RPG, Card Game...",http://steamcommunity.com/app/643980/reviews/?...,"[Single-player, Multi-player, Online Multi-Pla...",Free To Play,0.0,643980.0,Secret Level SRL
88312,Poolians.com,"[Casual, Free to Play, Indie, Simulation, Sports]",Real Pool 3D - Poolians,Real Pool 3D - Poolians,http://store.steampowered.com/app/670290/Real_...,2017-07-24,"[Free to Play, Simulation, Sports, Casual, Ind...",http://steamcommunity.com/app/670290/reviews/?...,"[Single-player, Multi-player, Online Multi-Pla...",Free to Play,0.0,670290.0,Poolians.com


In [34]:
# Ahora identifico la cantidad de valores nulos que qudaron en el dataset
for column in games.columns:
    print(f'Hay {games[column].isna().sum()} valores nulos en {column}')

Hay 8052 valores nulos en publisher
Hay 3283 valores nulos en genres
Hay 2 valores nulos en app_name
Hay 2050 valores nulos en title
Hay 0 valores nulos en url
Hay 2067 valores nulos en release_date
Hay 163 valores nulos en tags
Hay 2 valores nulos en reviews_url
Hay 670 valores nulos en specs
Hay 1377 valores nulos en price
Hay 0 valores nulos en early_access
Hay 2 valores nulos en id
Hay 3299 valores nulos en developer


In [35]:
# Sabiendo la cantidad de nulos que quedaron, ahora procedo a eliminarlos 
games = games.dropna() 
#Ahora hago el checkeo 
for column in games.columns:
    print(f'Hay {games[column].isna().sum()} valores nulos en {column}')

Hay 0 valores nulos en publisher
Hay 0 valores nulos en genres
Hay 0 valores nulos en app_name
Hay 0 valores nulos en title
Hay 0 valores nulos en url
Hay 0 valores nulos en release_date
Hay 0 valores nulos en tags
Hay 0 valores nulos en reviews_url
Hay 0 valores nulos en specs
Hay 0 valores nulos en price
Hay 0 valores nulos en early_access
Hay 0 valores nulos en id
Hay 0 valores nulos en developer


Borro las columnas que no voy a utilizar

In [36]:
games.drop(columns= ['app_name', 'url', 'tags', 'reviews_url', 'specs', 'early_access'], inplace=True)

In [37]:
games.info()

<class 'pandas.core.frame.DataFrame'>
Index: 22530 entries, 88310 to 120443
Data columns (total 7 columns):
 #   Column        Non-Null Count  Dtype  
---  ------        --------------  -----  
 0   publisher     22530 non-null  object 
 1   genres        22530 non-null  object 
 2   title         22530 non-null  object 
 3   release_date  22530 non-null  object 
 4   price         22530 non-null  object 
 5   id            22530 non-null  float64
 6   developer     22530 non-null  object 
dtypes: float64(1), object(6)
memory usage: 1.4+ MB


In [38]:
games.head(2)

,publisher,genres,title,release_date,price,id,developer
88310,Kotoshiro,"[Action, Casual, Indie, Simulation, Strategy]",Lost Summoner Kitty,2018-01-04,4.99,761140.0,Kotoshiro
88311,"Making Fun, Inc.","[Free to Play, Indie, RPG, Strategy]",Ironbound,2018-01-04,Free To Play,643980.0,Secret Level SRL


In [47]:
games['release_year'] = games['release_date'].str[:4]
games.drop(columns= ['release_date'], inplace=True)

In [48]:
games.head(2)

,publisher,genres,title,price,id,developer,release_year
88310,Kotoshiro,"[Action, Casual, Indie, Simulation, Strategy]",Lost Summoner Kitty,4.99,761140.0,Kotoshiro,2018
88311,"Making Fun, Inc.","[Free to Play, Indie, RPG, Strategy]",Ironbound,Free To Play,643980.0,Secret Level SRL,2018


Modifico los datos dentro de la columna price

In [49]:
# Identifico que datos hay dentro de la columna
games['price'].unique()

array([4.99, 'Free To Play', 'Free to Play', 0.99, 3.99, 18.99, 29.99,
       10.99, 2.99, 1.5899999999999999, 14.99, 1.99, 59.99, 9.99, 6.99,
       7.99, 39.99, 'Free', 19.99, 7.49, 8.99, 12.99, 5.99, 2.49, 15.99,
       1.25, 24.99, 17.99, 61.99, 3.49, 11.99, 13.99, 'Free Demo',
       'Play for Free!', 34.99, 1.49, 32.99, 99.99, 14.95, 69.99, 16.99,
       79.99, 49.99, 13.98, 149.99, 771.71, 'Install Now', 21.99, 89.99,
       'Play WARMACHINE: Tactics Demo', 0.98, 139.92, 4.29, 'Free Mod',
       54.99, 64.99, 74.99, 0.89, 0.5, 'Play Now', 299.99, 1.29, 119.99,
       44.99, 3.0, 15.0, 1.3900000000000001, 'Free HITMAN™ Holiday Pack',
       2.0, 4.0, 1.9500000000000002, 1.5, 6.66, 26.99, 399.99, 31.99,
       20.0, 40.0, 5.0, 3.33, 38.85, 71.7, 995.0, 5.49, 27.49, 3.39, 6.0,
       19.95, 20.99, 499.99, 27.99, 199.99, 4.68, 131.4, 44.98, 202.76,
       2.3, 0.9500000000000001, 36.99, 172.24, 249.99, 2.9699999999999998,
       10.96, 2.66, 6.48, 10.0, 1.0, 11.15, 'Play the Demo', 

In [50]:
def price_to_float(price):
    return float(price) if not isinstance(price, str) else 0

games['price'] = games['price'].apply(price_to_float)

In [51]:
games['price'] = games['price'].astype('float32')

In [52]:
#pd.set_option('display.float_format', '{:.2f}'.format)
games['price'].unique()

array([4.9900e+00, 0.0000e+00, 9.9000e-01, 3.9900e+00, 1.8990e+01,
       2.9990e+01, 1.0990e+01, 2.9900e+00, 1.5900e+00, 1.4990e+01,
       1.9900e+00, 5.9990e+01, 9.9900e+00, 6.9900e+00, 7.9900e+00,
       3.9990e+01, 1.9990e+01, 7.4900e+00, 8.9900e+00, 1.2990e+01,
       5.9900e+00, 2.4900e+00, 1.5990e+01, 1.2500e+00, 2.4990e+01,
       1.7990e+01, 6.1990e+01, 3.4900e+00, 1.1990e+01, 1.3990e+01,
       3.4990e+01, 1.4900e+00, 3.2990e+01, 9.9990e+01, 1.4950e+01,
       6.9990e+01, 1.6990e+01, 7.9990e+01, 4.9990e+01, 1.3980e+01,
       1.4999e+02, 7.7171e+02, 2.1990e+01, 8.9990e+01, 9.8000e-01,
       1.3992e+02, 4.2900e+00, 5.4990e+01, 6.4990e+01, 7.4990e+01,
       8.9000e-01, 5.0000e-01, 2.9999e+02, 1.2900e+00, 1.1999e+02,
       4.4990e+01, 3.0000e+00, 1.5000e+01, 1.3900e+00, 2.0000e+00,
       4.0000e+00, 1.9500e+00, 1.5000e+00, 6.6600e+00, 2.6990e+01,
       3.9999e+02, 3.1990e+01, 2.0000e+01, 4.0000e+01, 5.0000e+00,
       3.3300e+00, 3.8850e+01, 7.1700e+01, 9.9500e+02, 5.4900e

In [53]:
games.info()

<class 'pandas.core.frame.DataFrame'>
Index: 22530 entries, 88310 to 120443
Data columns (total 7 columns):
 #   Column        Non-Null Count  Dtype  
---  ------        --------------  -----  
 0   publisher     22530 non-null  object 
 1   genres        22530 non-null  object 
 2   title         22530 non-null  object 
 3   price         22530 non-null  float32
 4   id            22530 non-null  float64
 5   developer     22530 non-null  object 
 6   release_year  22530 non-null  object 
dtypes: float32(1), float64(1), object(5)
memory usage: 1.3+ MB


In [54]:
# Convertir todas las columnas de tipo 'object' a 'string'
for column in games.select_dtypes(include=['object']).columns:
    games[column] = games[column].astype(str)

Reducir el archivo para que la carga del render no se me complique 

In [55]:
games.shape

(22530, 7)

In [56]:
games.to_csv('../Dataset/games.csv')

In [57]:
games.to_parquet('../Dataset/games_clean.parquet')